### 这里使用pycharm 多核运算好的数据，
    分三个训练集，一个给 特征 1,2,4用，一个专门给 3用，一个专门给 5用
     并且使用 10% 的阈值，即 3850 和 删掉训练标签集中 abnormal 的
     整个程序耗时 3.5mins 只有两个输入csv，设定相关阈值后 直接输出分数

In [36]:
import numpy as np
import pandas as pd

# 子函数，用于删除从 csv 中读入文件的第一行
def func_delete_Unnamed0(df_from_csv):
    df_delete_Unnamed0 =df_from_csv.drop(['Unnamed: 0'],axis=1)
    return df_delete_Unnamed0

# 函数功能：第一次筛选，筛选特征值，只保留对应一个特征，其数据值超过阈值的特征
# 输入：df_all_feature ：待处理的数据集，包含较多特征，需要减少特征
#       num_threshold_feature ：特征值对应的数据个数的阈值
def select_feature_via_threshold(df_all_feature,num_threshold_feature):
    num_rows = df_all_feature.shape[0]
    num_columns =  df_all_feature.shape[1]
    # 输入的数据集的特征名称列表
    list_feature_names = df_all_feature.columns.tolist()
    # 特征对应数据值超过阈值的特征名称 组成的列表
    list_feature_names_after_threshold = []
    for i in range(num_columns):
        if df_all_feature.iloc[:,i].count() >= num_threshold_feature:
            list_feature_names_after_threshold.append(list_feature_names[i])
   
    df_selected_via_feature_threshold = df_all_feature[list_feature_names_after_threshold]
    return df_selected_via_feature_threshold  

# 38199 中删除 49个不良数据
# 清洗数据，看是否有重复数据
# 经过验证，没有重复数据，TODO：func_delete_repeat_cows_df 添加一个返回值，返回由重复元素组成的df
# 子函数，删除列表中重复元素，还是原始顺序
# 删除 dataframe 有简单方法 df.drop(df.index[[1,3]])
def func_delete_repeat_element_list(list_with_repeat):
    list_without_repeat = list(set(list_with_repeat))
    list_without_repeat.sort(key=list_with_repeat.index)
    return list_without_repeat

def func_delete_repeat_cows_df(df,str_id_name):
    list_id_names = df[str_id_name].tolist()
    list_id_names_without_repeat = func_delete_repeat_element_list(list_id_names)
    df_without_repeat = df[df[str_id_name].isin(list_id_names_without_repeat)]
    return df_without_repeat

# 筛选出不良数据
# 子函数，删除列表中重复元素，还是原始顺序
def func_delete_repeat_element_list(list_with_repeat):
    list_without_repeat = list(set(list_with_repeat))
    list_without_repeat.sort(key=list_with_repeat.index)
    return list_without_repeat

def func_judge_isnot_change_str(check_str):
    try:
        float(check_str)
        return False
    except ValueError:
        return True
    
# 行的循环改为通过 index 的列表循环
# 为了精准，返回值是 id号组成的列表
def func_select_sample_cannot_change_str(df_under):
    list_ID_cannot_change_str = []
    list_index_cannot_change_str = []
    list_index = df_under.index.tolist()
    for i in list_index:
       # 这里'vid' 特征是 str类型，但是要保存'vid'特征，'vid'特征是第0列，所以要从1开始循环
        for j in range(1,df_under.shape[1]):
            if func_judge_isnot_change_str( df_under.loc[i].values[j]) == True:
                list_index_cannot_change_str.append(i)
            
    list_index_cannot_change_str_without_repeat = func_delete_repeat_element_list(list_index_cannot_change_str)
    df_cannot_change_str = df_under.loc[list_index_cannot_change_str_without_repeat]
    return df_cannot_change_str

#删除 标签训练集中的不良数据
def func_delete_df(df_under_deleted,df_deleter,str_ID_name):
    list_under_deleted = df_under_deleted[str_ID_name].tolist()
    list_deleter = df_deleter[str_ID_name].tolist()
    list_difference =  list(set(list_under_deleted) ^ set(list_deleter))
    df_result = df_under_deleted[df_under_deleted[str_ID_name].isin(list_difference)]
    return df_result

# 功能：从全体样本中筛选出部分样本
# str_feature_type：用于筛选的特征值
def select_sample_has_label(df_all_sample,df_part_sample,str_feature_type):
    # 获取标签训练集的 ID list  即 vid 的 list
    list_vid_train_label =df_part_sample[str_feature_type].tolist()
    # 根据ID号，筛选出38199个样本，再组成一个新的 dataframe
    df_selected_via_vid = df_all_sample.loc[df_all_sample[str_feature_type].isin(list_vid_train_label)]
    # 之前的 index 还是断断续续的，扔掉之前的，重新从0开始排序
    df_selected_via_vid_index_resetted = df_selected_via_vid.reset_index(drop=True)
    return df_selected_via_vid_index_resetted

# 训练特征集重新排序
def func_sort_df(df_under_sorted,df_model,str_sort_key):
    list_sorter=  df_model[str_sort_key].tolist()
    dict_sorterIndex = dict(zip(list_sorter,range(len(list_sorter))))
    str_sort_key_rank = str_sort_key + '_rank'
    df_under_sorted[str_sort_key_rank] = df_under_sorted[str_sort_key].map(dict_sorterIndex)
    df_under_sorted.sort_values([str_sort_key_rank], \
        ascending = [True ], inplace = True)
    df_under_sorted.drop(str_sort_key_rank, 1, inplace = True)
    #df_sorted = df_under_sorted[df_under_sorted[str_sort_key].isin(list_model_sort_key)]
    return df_under_sorted

# 删除 df总对应vid的list的样本
def func_delete_df_by_list(df_under_deleted,list_deleter,str_ID_name):
    list_under_deleted = df_under_deleted[str_ID_name].tolist()
    list_difference =  list(set(list_under_deleted) ^ set(list_deleter))
    df_result = df_under_deleted[df_under_deleted[str_ID_name].isin(list_difference)]
    return df_result
# 转 flaot 类型子函数
def change_not_float_to_nan(check_str):
    try:
        float(check_str)
        str_c = float(check_str)
        return str_c
    except ValueError:
        check_str = np.nan
        return check_str

def func_str_to_float_df(df_str):
    df_str=df_str.reset_index(drop=True)
    df_float = df_str
    for i in range(df_str.shape[0]):
        # 这里'vid' 特征是 str类型，但是要保存'vid'特征，'vid'特征是第0列，所以要从1开始循环
        for j in range(1,df_str.shape[1]):
            df_float.iat[i, j] = change_not_float_to_nan(df_str.loc[i].values[j])
    return df_float 
    
# 筛选范围异常值的子函数，用于判断标签训练集中 如血压值是否在合理范围内
# 输出4个list
def func_select_abnormal_values(df_label,
                                num_low_threshold_abnormal,
                                num_high_threshold_abnormal,
                                num_squence_label,
                                num_squence_ID):
    list_index = df_label.index.tolist()
    
    list_low_value_abnormal = []
    list_high_value_abnormal = []
    list_low_vid_abnormal = []
    list_high_vid_abnormal = []
    
    for i in list_index:
        #print('type',type(df_label.loc[i].values[num_squence_label]))
        if df_label.loc[i].values[num_squence_label] < num_low_threshold_abnormal:
            list_low_value_abnormal.append(df_label.loc[i].values[num_squence_label])
            list_low_vid_abnormal.append(df_label.loc[i].values[num_squence_ID])
    
    for i in list_index:
        if df_label.loc[i].values[num_squence_label] > num_high_threshold_abnormal:
            list_high_value_abnormal.append(df_label.loc[i].values[num_squence_label])
            list_high_vid_abnormal.append(df_label.loc[i].values[num_squence_ID])
    
    return list_low_value_abnormal,\
            list_high_value_abnormal,\
            list_low_vid_abnormal,\
            list_high_vid_abnormal
            
# 列表合并子函数
def func_combine_list(list_a,list_b):
    list_result =list_a + list_b
    return list_result

# 判断舒张压 是否 小于 收缩压 子函数
def func_select_abnormal_relationship_two_labels(df_label,
                                                num_squence_label_A,
                                                num_squence_label_B,
                                                 num_squence_ID):
    list_index = df_label.index.tolist()
    
    list_value_abnormal_relationship_pair = []
    list_vid_abnormal_relationship = []
    
    for i in list_index:
        if df_label.loc[i].values[num_squence_label_A] <= df_label.loc[i].values[num_squence_label_B]:
            list_value_abnormal_relationship_pair.append(df_label.loc[i].values[num_squence_label_A])
            list_value_abnormal_relationship_pair.append(df_label.loc[i].values[num_squence_label_B])
            list_value_abnormal_relationship_pair.append(',')
            list_vid_abnormal_relationship.append(df_label.loc[i].values[num_squence_ID])
    return  list_value_abnormal_relationship_pair,\
            list_vid_abnormal_relationship

# 查找并处理标签训练集中异常值
def func_delete_abnormal_value(df_under):
    # 查找 收缩压中的异常值
    list_low_SBP_abnormal,list_high_SBP_abnormal,list_vid_low_SBP_abnormal,list_vid_high_SBP_abnormal =  func_select_abnormal_values(df_label = df_under,
                                                                                            num_low_threshold_abnormal = 50,
                                                                                            num_high_threshold_abnormal = 300,
                                                                                            num_squence_label = 1,
                                                                                            num_squence_ID = 0)
    # 合并2种异常值vid列表
    list_vid_SBP_abnormal = func_combine_list(list_vid_low_SBP_abnormal,list_vid_high_SBP_abnormal)  
    # 查找舒张压（DBP）异常值
    list_low_DBP_abnormal,list_high_DBP_abnormal,list_vid_low_DBP_abnormal,list_vid_high_DBP_abnormal =  func_select_abnormal_values(df_label =df_under,
                                                                                            num_low_threshold_abnormal = 30,
                                                                                            num_high_threshold_abnormal = 200,
                                                                                            num_squence_label = 2,
                                                                                            num_squence_ID = 0)
    list_vid_DBP_abnormal = func_combine_list(list_vid_low_DBP_abnormal,list_vid_high_DBP_abnormal)  
    # 判断收缩压与舒张压关系异常的样本
    list_value_abnormal_SBP_and_DBP ,  list_vid_abnormal_SBP_and_DBP = func_select_abnormal_relationship_two_labels(df_label = df_under,
                                                                                                               num_squence_label_A = 1,
                                                                                                               num_squence_label_B = 2,
                                                                                                               num_squence_ID = 0)
    # 查看血清甘油三脂（TG）的 异常值
    # 都属于正常值
    list_low_TG_abnormal,list_high_TG_abnormal,list_vid_low_TG_abnormal,list_vid_high_TG_abnormal =  func_select_abnormal_values(df_label = df_under,
                                                                                            num_low_threshold_abnormal = 0.1,
                                                                                            #num_high_threshold_abnormal = 16.5,
                                                                                            num_high_threshold_abnormal = 5,
                                                                                            num_squence_label = 3,
                                                                                            num_squence_ID = 0)
    list_vid_TG_abnormal = func_combine_list(list_vid_low_TG_abnormal,list_vid_high_TG_abnormal)  
    # 查看血清高密度脂蛋白（HDL）的 异常值
    list_low_HDL_abnormal,list_high_HDL_abnormal,list_vid_low_HDL_abnormal,list_vid_high_HDL_abnormal =  func_select_abnormal_values(df_label = df_under,
                                                                                            num_low_threshold_abnormal = 0.5,
                                                                                            num_high_threshold_abnormal = 5,
                                                                                            num_squence_label = 4,
                                                                                            num_squence_ID = 0)
    list_vid_HDL_abnormal = func_combine_list(list_vid_low_HDL_abnormal,list_vid_high_HDL_abnormal)  
    # 查看血清低密度脂蛋白（LDL）的 异常值
    # 结果中竟然有类负数，这些数据要删除
    list_low_LDL_abnormal,list_high_LDL_abnormal,list_vid_low_LDL_abnormal,list_vid_high_LDL_abnormal =  func_select_abnormal_values(df_label = df_under,
                                                                                            #num_low_threshold_abnormal = 0,
                                                                                            num_low_threshold_abnormal = 0,
                                                                                            #num_high_threshold_abnormal = 12,
                                                                                            num_high_threshold_abnormal = 12,
                                                                                            num_squence_label = 5,
                                                                                            num_squence_ID = 0)
    list_vid_LDL_abnormal = func_combine_list(list_vid_low_LDL_abnormal,list_vid_high_LDL_abnormal) 
    # 总的 异常标签值 对应的vid列表
    list_vid_three_abnormal = list_vid_SBP_abnormal +\
                                list_vid_DBP_abnormal +\
                                list_vid_HDL_abnormal +\
                                list_vid_abnormal_SBP_and_DBP
    # 去除重复数据
    list_vid_three_abnormal = func_delete_repeat_element_list(list_vid_three_abnormal)
    list_vid_TG_abnormal = func_delete_repeat_element_list(list_vid_TG_abnormal)
    list_vid_LDL_abnormal = func_delete_repeat_element_list(list_vid_LDL_abnormal)
    return list_vid_three_abnormal,list_vid_TG_abnormal, list_vid_LDL_abnormal
            
# 通过写入再读取的方式 讲str类型转换为float类型    
def func_df_str_to_float_by_read_csv(df_str):
    df_str.to_csv('temp_str_to_float.csv',index = False)
    df_float = pd.read_csv('temp_str_to_float.csv')
    return df_float
   

def print_df_info(df):
    print("\n", "\n", df.shape) 
    print("\n","\n",df.head(5))



In [37]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import math

if __name__ == "__main__":
    # 读取 pycharm 多核运算 根据 10%为阈值筛选出的 数据 [38199 * 286]
    dataset_train_feature_all_float_read = pd.read_csv('dataset_train_feature_all_float_ten_per.csv')
    # 导入标签数据
    dataset_train_label= pd.read_csv("meinian_round1_train_20180408.csv", encoding="gb2312")  
    # 删除第一列的index
    dataset_train_feature_all_float = func_delete_Unnamed0(dataset_train_feature_all_float_read)
    # 第二次筛选
    num_threshold_feature = 3850
    dataset_train_feature_selected_by_threshold_second = select_feature_via_threshold(dataset_train_feature_all_float,num_threshold_feature)
    # 清洗数据，看是否有重复数据
    dataset_train_label = func_delete_repeat_cows_df(dataset_train_label,'vid')
    # 筛选出不良数据
    dataset_train_label_cannot_change_str = func_select_sample_cannot_change_str(dataset_train_label)
    #删除 标签训练集中的不良数据
    # TODO 时间有限，这里应该 把'>6.22' 这里的数据提取出 6.22 的，这里不做处理，直接删除这样的数据
    dataset_train_label_cannot_change_str_deleted = func_delete_df(dataset_train_label,
                                                               dataset_train_label_cannot_change_str,
                                                              'vid')
    ## 删除特征训练集中 对应标签训练集中的不良数据
    dataset_train_feature_cannot_change_str_deleted = func_delete_df(dataset_train_feature_selected_by_threshold_second,
                                                                dataset_train_label_cannot_change_str,
                                                                'vid')
   
    # 通过写入再读取的方式 讲str类型转换为float类型
    dataset_train_label_cannot_change_str_deleted = func_df_str_to_float_by_read_csv(dataset_train_label_cannot_change_str_deleted )

    print ("\n", "dataset_train_label_cannot_change_str_deleted.shape: ","\n",
           dataset_train_label_cannot_change_str_deleted.shape)

    
    


 dataset_train_label_cannot_change_str_deleted.shape:  
 (38150, 6)


In [40]:
# 计算所有 标签训练集中 abnormal 数据对应 vid 的list
    #list_vid_five_abnormal = func_delete_abnormal_value(dataset_train_label_cannot_change_str_deleted)
    # 再次删除 标签训练集中 abnormal 数据
   
list_vid_three_abnormal,list_vid_TG_abnormal,list_vid_LDL_abnormal = func_delete_abnormal_value(dataset_train_label_cannot_change_str_deleted)
    # 再次删除 标签训练集中 abnormal 数据
dataset_train_label_three_abnormal_deleted =  func_delete_df_by_list(dataset_train_label_cannot_change_str_deleted,
                                                                         list_vid_three_abnormal,
                                                                         'vid')
dataset_train_label_TG_abnormal_deleted =  func_delete_df_by_list(dataset_train_label_cannot_change_str_deleted,
                                                                      list_vid_TG_abnormal,
                                                                      'vid')
dataset_train_label_LDL_abnormal_deleted =  func_delete_df_by_list(dataset_train_label_cannot_change_str_deleted,
                                                                       list_vid_LDL_abnormal,
                                                                       'vid')
    ## 第二次 删除，删除 训练特征集中 对应 训练标签集中的 abnormal 数据
dataset_train_feature_three_abnormal_deleted = select_sample_has_label(dataset_train_feature_cannot_change_str_deleted,
                                                      dataset_train_label_three_abnormal_deleted,
                                                       'vid')
dataset_train_feature_TG_abnormal_deleted = select_sample_has_label(dataset_train_feature_cannot_change_str_deleted,
                                                      dataset_train_label_TG_abnormal_deleted,
                                                       'vid')
dataset_train_feature_LDL_abnormal_deleted = select_sample_has_label(dataset_train_feature_cannot_change_str_deleted,
                                                      dataset_train_label_LDL_abnormal_deleted,
                                                       'vid')
    # 训练特征集重新排序
dataset_train_feature_three_sorted_by_dataset_label = func_sort_df(dataset_train_feature_three_abnormal_deleted,
                                                             dataset_train_label_three_abnormal_deleted,
                                                             'vid')
dataset_train_feature_TG_sorted_by_dataset_label = func_sort_df(dataset_train_feature_TG_abnormal_deleted,
                                                             dataset_train_label_TG_abnormal_deleted,
                                                             'vid')
dataset_train_feature_LDL_sorted_by_dataset_label = func_sort_df(dataset_train_feature_LDL_abnormal_deleted,
                                                             dataset_train_label_LDL_abnormal_deleted,
                                                             'vid')
dataset_train_feature_three_sorted_by_dataset_label.to_csv('dataset_train_feature_three_20180507.csv',index = False)
dataset_train_feature_TG_sorted_by_dataset_label.to_csv('dataset_train_feature_TG_20180507.csv',index = False)
dataset_train_feature_LDL_sorted_by_dataset_label.to_csv('dataset_train_feature_LDL_20180507.csv',index = False)

dataset_train_label_three_abnormal_deleted.to_csv('dataset_train_label_three_20180507.csv',index = False)
dataset_train_label_TG_abnormal_deleted.to_csv('dataset_train_label_TG_20180507.csv',index = False)
dataset_train_label_LDL_abnormal_deleted.to_csv('dataset_train_label_LDL_20180507.csv',index = False)


print ("\n", " dataset_train_feature_three_sorted_by_dataset_label.shape: ","\n",
            dataset_train_feature_three_sorted_by_dataset_label.shape)
print ("\n", " dataset_train_label_three_abnormal_deleted.shape: ","\n",
            dataset_train_label_three_abnormal_deleted.shape)
    
print ("\n", " dataset_train_feature_TG_sorted_by_dataset_label.shape: ","\n",
            dataset_train_feature_TG_sorted_by_dataset_label.shape)
print ("\n", " dataset_train_label_TG_abnormal_deleted.shape: ","\n",
            dataset_train_label_TG_abnormal_deleted.shape)
    
print ("\n", " dataset_train_feature_LDL_sorted_by_dataset_label.shape: ","\n",
            dataset_train_feature_LDL_sorted_by_dataset_label.shape)
print ("\n", " dataset_train_label_LDL_abnormal_deleted.shape: ","\n",
            dataset_train_label_LDL_abnormal_deleted.shape)

    



  dataset_train_feature_three_sorted_by_dataset_label.shape:  
 (38144, 136)

  dataset_train_label_three_abnormal_deleted.shape:  
 (38144, 6)

  dataset_train_feature_TG_sorted_by_dataset_label.shape:  
 (37353, 136)

  dataset_train_label_TG_abnormal_deleted.shape:  
 (37353, 6)

  dataset_train_feature_LDL_sorted_by_dataset_label.shape:  
 (38147, 136)

  dataset_train_label_LDL_abnormal_deleted.shape:  
 (38147, 6)


In [41]:
# 预测结果和计算得分
def func_predict_and_score_XGBRegressor(df_X,df_Y):
    X = df_X.iloc[:,1:]
    array_X = np.array(X)
    array_Y = np.array(df_Y)
    seed=  7
    test_size = 0.33
    #test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(array_X,
                                                        array_Y, 
                                                        test_size=test_size,
                                                        random_state=seed)
    model = XGBRegressor()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
   
    num_people = len(y_pred)
    sum_score = 0
    score = 0
    for j in range(len(y_pred)):
        sum_score = sum_score + (math.log(y_pred[j]+1) - math.log(y_test[j]+1)) ** 2
    score = sum_score / num_people
    list_ypred_and_score = []
    list_ypred_and_score.append(y_pred)
    list_ypred_and_score.append(score)
    #list_ypred_and_score[0] = y_pred
    #list_ypred_and_score[1] = score
    #print ("\n", "y_pred: ","\n",y_pred)
    return list_ypred_and_score 
# 训练模型，计算得分
    # 预处理好的训练特征集
dataset_train_feature_preprocessed_Three = dataset_train_feature_three_sorted_by_dataset_label
dataset_train_feature_preprocessed_TG = dataset_train_feature_TG_sorted_by_dataset_label
dataset_train_feature_preprocessed_LDL = dataset_train_feature_LDL_sorted_by_dataset_label
    # 预处理好的训练标签集
dataset_train_label_Three = dataset_train_label_three_abnormal_deleted
dataset_train_label_TG = dataset_train_label_TG_abnormal_deleted
dataset_train_label_LDL = dataset_train_label_LDL_abnormal_deleted

#X = dataset_train_feature_preprocessed.iloc[:,1:]
X_three = dataset_train_feature_preprocessed_Three.iloc[:,1:]
X_TG = dataset_train_feature_preprocessed_TG.iloc[:,1:]
X_LDL = dataset_train_feature_preprocessed_LDL.iloc[:,1:]

Y_1 = dataset_train_label_Three.iloc[:,1]
Y_2 = dataset_train_label_Three.iloc[:,2]
Y_4 = dataset_train_label_Three.iloc[:,4]

Y_3 = dataset_train_label_TG.iloc[:,3]
Y_5 = dataset_train_label_LDL.iloc[:,5]

score_1 = func_predict_and_score_XGBRegressor(X_three,Y_1)[1]
score_2 = func_predict_and_score_XGBRegressor(X_three,Y_2)[1]
score_3 = func_predict_and_score_XGBRegressor(X_TG,Y_3)[1]
score_4 = func_predict_and_score_XGBRegressor(X_three,Y_4)[1]
score_5 = func_predict_and_score_XGBRegressor(X_LDL,Y_5)[1]

# 计算得分子函数
score_mean  = (score_1 + score_2 + score_3 + score_4 + score_5) / 5
print ("\n", "score_1: ","\n",score_1)
print ("\n", "score_2: ","\n",score_2)
print ("\n", "score_3: ","\n",score_3)
print ("\n", "score_4: ","\n",score_4)
print ("\n", "score_5: ","\n",score_5)
print ("\n", "score_mean: ","\n",score_mean)



 score_1:  
 0.016163520030159778

 score_2:  
 0.019717984561833612

 score_3:  
 0.059463257350289796

 score_4:  
 0.013740588374619958

 score_5:  
 0.03755522946777588

 score_mean:  
 0.029328115956935807


## 进入模型训练部分